In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
import torch.optim as optim
from torch.autograd import Variable
import time
from KL_PMVAE import KL_PMVAE_2omics, KL_PMVAE_genes, KL_PMVAE_mirnas
from utils import sort_data, load_data, load_pathway, bce_recon_loss, kl_divergence, get_match_id
from train_KL_PMVAE import train_KL_PMVAE

dtype = torch.FloatTensor


# In[ ]:


In [2]:
import os

os.chdir('/cloud-home/U1039935/Autosurv/autosurv')

In [3]:
"""
Example listed here is for TCGA-BRCA data. If using TCGA-OV data, please adjust the parameters/covariates accordingly
"""

input_n1 = 2699
input_n2 = 516
z_dim = [8, 16, 32, 64]
EPOCH_NUM = [800, 1200, 1600, 2000, 2400]
NUM_CYCLES = [2, 4, 5, 10]
Initial_Learning_Rate = [0.1, 0.05, 0.005, 0.001]
L2_Lambda = [0.1, 0.05, 0.005, 0.0005]
CUTTING_RATIO = [0.3, 0.5, 0.7, 0.9]
# EPOCH_NUM = [800, 1200, 1600]
# NUM_CYCLES = [2, 4]
# Initial_Learning_Rate = [0.1, 0.01]
# L2_Lambda = [0.1]
patient_id_train, x_train_gene, ytime_train, yevent_train, age_train, stage_i_train, stage_ii_train, race_white_train = load_data("processed_data_example/TCGA_BRCA/tune/minmax_normalized/data_train_gene_minmax_tune.csv", dtype)
patient_id_valid, x_valid_gene, ytime_valid, yevent_valid, age_valid, stage_i_valid, stage_ii_valid, race_white_valid = load_data("processed_data_example/TCGA_BRCA/tune/minmax_normalized/data_valid_gene_minmax_tune.csv", dtype)
pathway_mask_tune = load_pathway("processed_data_example/TCGA_BRCA/tune/minmax_normalized/pathway_mask.csv", dtype)

_, x_train_mirna, _, _, _, _, _, _ = load_data("processed_data_example/TCGA_BRCA/tune/minmax_normalized/data_train_mirna_minmax_tune.csv", dtype)
_, x_valid_mirna, _, _, _, _, _, _ = load_data("processed_data_example/TCGA_BRCA/tune/minmax_normalized/data_valid_mirna_minmax_tune.csv", dtype)



patient_id_train_overall, x_train_gene_overall, ytime_train_overall, yevent_train_overall, age_train_overall, stage_i_train_overall, stage_ii_train_overall, race_white_train_overall = load_data("processed_data_example/TCGA_BRCA/train_test_split/minmax_normalized/data_train_gene_minmax_overall.csv", dtype)
patient_id_test_overall, x_test_gene_overall, ytime_test_overall, yevent_test_overall, age_test_overall, stage_i_test_overall, stage_ii_test_overall, race_white_test_overall = load_data("processed_data_example/TCGA_BRCA/train_test_split/minmax_normalized/data_test_gene_minmax_overall.csv", dtype)
pathway_mask_test = load_pathway("processed_data_example/TCGA_BRCA/train_test_split/minmax_normalized/pathway_mask.csv", dtype)

_, x_train_mirna_overall, _, _, _, _, _, _ = load_data("processed_data_example/TCGA_BRCA/train_test_split/minmax_normalized/data_train_mirna_minmax_overall.csv", dtype)
_, x_test_mirna_overall, _, _, _, _, _, _ = load_data("processed_data_example/TCGA_BRCA/train_test_split/minmax_normalized/data_test_mirna_minmax_overall.csv", dtype)


## Explore data

In [26]:
x_train_mirna.shape

torch.Size([669, 516])

In [27]:
x_train_gene.shape

torch.Size([669, 2699])

In [28]:
x_train_gene_overall.shape

torch.Size([857, 2699])

In [29]:
x_test_gene_overall.shape

torch.Size([201, 2699])

In [30]:
pathway_mask_test.shape

torch.Size([581, 2699])

In [31]:
import pandas as pd

df = pd.read_csv("processed_data_example/TCGA_BRCA/tune/minmax_normalized/pathway_mask.csv")

In [32]:
df.head(5)

,Unnamed: 0,R-HSA-1989781,R-HSA-418594,R-HSA-975298,R-HSA-418555,R-HSA-1442490,R-HSA-2565942,R-HSA-380259,R-HSA-380270,R-HSA-438066,...,R-HSA-373752,R-HSA-3000480,R-HSA-196299,R-HSA-3322077,R-HSA-1251985,R-HSA-5339716,R-HSA-5358747,R-HSA-5358749,R-HSA-5358751,R-HSA-5358752
0,ENSG00000000938,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ENSG00000000971,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ENSG00000001461,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ENSG00000001630,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ENSG00000003989,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
import pandas as pd

df = pd.read_csv("processed_data_example/TCGA_BRCA/tune/minmax_normalized/pathway_mask.csv")
selected_columns = df.filter(like='1251999', axis=1)


print(selected_columns)

Empty DataFrame
Columns: []
Index: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, ...]

[2699 rows x 0 columns]


In [34]:
pathway_mask_tune

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [35]:
pathway_mask_test.sum(1)

tensor([48., 87.,  3., 53., 37., 36., 25., 28.,  5.,  3.,  5.,  7., 35., 43.,
        45., 25., 10., 15.,  6., 19.,  5., 14., 10., 10.,  1., 14., 13.,  8.,
        12.,  8.,  9., 16., 18., 13.,  6., 20., 16., 41., 19., 13., 38., 14.,
        23., 34., 13.,  9., 16., 22., 16., 20., 16., 37., 35.,  8.,  5., 50.,
        67., 18., 24., 16.,  5.,  8.,  6.,  6.,  6.,  5.,  6., 11., 14., 37.,
         7.,  7.,  8.,  6., 30., 26., 29.,  8., 31., 58., 10., 15., 16., 14.,
        11., 18., 17., 31., 35., 52.,  3., 12., 33., 66., 15., 27.,  8., 41.,
        19., 33., 10.,  9.,  5.,  3., 12.,  7., 14.,  5.,  7., 12., 31.,  7.,
         7., 35.,  8., 10., 10., 16., 23., 31., 13., 32., 14., 44., 22., 13.,
         9., 14., 10., 18.,  8., 16., 36.,  5.,  6., 17.,  8., 44.,  8., 14.,
        14., 14.,  9., 10., 27., 14.,  7.,  6.,  8.,  2., 21.,  8.,  7., 11.,
        10.,  6., 23., 14.,  8., 11., 14., 14.,  7., 13.,  8., 43., 19.,  8.,
        12., 12., 35.,  7., 42., 15., 11., 37.,  6., 11.,  9., 1

In [36]:
x_train_gene.shape

torch.Size([669, 2699])

In [37]:
x_train_gene_overall.shape

torch.Size([857, 2699])

In [38]:
print(pathway_mask_tune.shape, pathway_mask_test.shape)

torch.Size([581, 2699]) torch.Size([581, 2699])


## model

They first use tuned data(80% of the whole data) to find the optim parameter

In [4]:
import time


In [ ]:
%%capture output
# %%time

start_time = time.time()

opt_l2 = 0.
opt_lr = 0.
opt_dim = 0
opt_epoch_num = 0.
opt_num_cycle = 0.
opt_cr = 0.
opt_loss = torch.Tensor([float("Inf")])
if torch.cuda.is_available():
    opt_loss = opt_loss.cuda()
for l2 in L2_Lambda:
    for lr in Initial_Learning_Rate:
        for Z in z_dim:
            for Epoch_num in EPOCH_NUM:
                for Num_cycles in NUM_CYCLES:
                    for cutting_ratio in CUTTING_RATIO:
                        _, _, _, _, train_loss_unsup, eval_loss_unsup = train_KL_PMVAE(x_train_gene, x_train_mirna, x_valid_gene, x_valid_mirna,
                                                                                       Z, input_n1, input_n2, pathway_mask_tune,
                                                                                       lr, l2, cutting_ratio, Epoch_num, Num_cycles, dtype,
                                                                                       path = "saved_models/unsup_checkpoint_tune.pt")
            
                        if eval_loss_unsup < opt_loss:
                            opt_l2 = l2
                            opt_lr = lr
                            opt_dim = Z
                            opt_epoch_num = Epoch_num
                            opt_num_cycle = Num_cycles
                            opt_cr = cutting_ratio
                            opt_loss = eval_loss_unsup
                        print("num_epoch: %s," %Epoch_num, "num_cycles: %s," %Num_cycles, "cutting_ratio: %s." %cutting_ratio)
                        print("L2: %s," %l2, "LR: %s," %lr, "z_dim: %s," %Z, "loss in validation: %s," %np.array(eval_loss_unsup.detach().cpu().numpy()).round(4), "loss in training: %s." %np.array(train_loss_unsup.detach().cpu().numpy()).round(4))

end_time = time.time()
print("Optimal num epoch: %s," %opt_epoch_num, "optimal num cycles: %s," %opt_num_cycle, "optimal cutting ratio: %s." %opt_cr)
print("Optimal L2: %s," %opt_l2, "optimal LR: %s," %opt_lr, "optimal z_dim: %s." %opt_dim)
print("--- %s seconds ---" % (end_time - start_time))

In [16]:
print("Optimal num epoch: %s," %opt_epoch_num, "optimal num cycles: %s," %opt_num_cycle, "optimal cutting ratio: %s." %opt_cr)
print("Optimal L2: %s," %opt_l2, "optimal LR: %s," %opt_lr, "optimal z_dim: %s." %opt_dim)
print("--- %s seconds ---" % (time.time() - start_time))

Optimal num epoch: 2400, optimal num cycles: 10, optimal cutting ratio: 0.5.
Optimal L2: 0.005, optimal LR: 0.005, optimal z_dim: 16.
--- 46159.66844058037 seconds ---


In [ ]:
#12.82hours

In [26]:
output.show()

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)

/cloud-home/U1039935/Autosurv/autosurv/KL_PMVAE.py:58: UserWarning: The torch.cuda.*DtypeTensor constructors are no longer recommended. It's best to use methods such as torch.tensor(data, dtype=*, device='cuda') to create tensors. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:78.)
  z = Variable(torch.cuda.FloatTensor(std.size()).normal_(0, 1))


In [9]:
train_mean, train_logvar, test_mean, test_logvar, train_loss_unsup, test_loss_unsup = train_KL_PMVAE(x_train_gene_overall, x_train_mirna_overall, x_test_gene_overall, x_test_mirna_overall,
                                                                                                     opt_dim, input_n1, input_n2, pathway_mask_test,
                                                                                                     opt_lr, opt_l2, opt_cr, opt_epoch_num, opt_num_cycle, dtype, save_model = True,
                                                                                                     path = "saved_models/unsup_checkpoint_overall.pt")
print("Loss in testing: %s," %np.array(test_loss_unsup.detach().cpu().numpy()).round(4), "loss in training: %s." %np.array(train_loss_unsup.detach().cpu().numpy()).round(4))


Epoch: 100, Loss in training: [2051.3591], loss in validation: [2070.5073].
Epoch: 200, Loss in training: [1964.2042], loss in validation: [1986.018].
Epoch: 300, Loss in training: [1920.1187], loss in validation: [1944.1882].
Epoch: 400, Loss in training: [1910.1398], loss in validation: [1937.0376].
Epoch: 500, Loss in training: [1887.346], loss in validation: [1912.845].
Epoch: 600, Loss in training: [1894.7974], loss in validation: [1923.6282].
Epoch: 700, Loss in training: [1891.115], loss in validation: [1920.3354].
Epoch: 800, Loss in training: [1883.998], loss in validation: [1912.7388].
Epoch: 900, Loss in training: [1887.68], loss in validation: [1916.9261].
Epoch: 1000, Loss in training: [1875.2795], loss in validation: [1903.9664].
Epoch: 1100, Loss in training: [1886.0614], loss in validation: [1915.3828].
Epoch: 1200, Loss in training: [1885.7174], loss in validation: [1915.2408].
Epoch: 1300, Loss in training: [1882.9482], loss in validation: [1912.4562].
Epoch: 1400, Lo

Create the data for LFSurv model (tune data & Overall data)

In [ ]:
tr_z = train_mean
tes_z = test_mean

print("Training sample size: %s," %tr_z.size()[0], "testing sample size: %s." %tes_z.size()[0])

In [ ]:
processed_tr_pre = torch.cat((ytime_train_overall, yevent_train_overall, age_train_overall, stage_i_train_overall, stage_ii_train_overall, race_white_train_overall, tr_z), 1)
processed_tes_pre = torch.cat((ytime_test_overall, yevent_test_overall, age_test_overall, stage_i_test_overall, stage_ii_test_overall, race_white_test_overall, tes_z), 1)

z_count = np.array(list(range(1, tr_z.size()[1]+1, 1))).astype('str')
z_names = np.char.add('Z_', z_count).tolist()

processed_tr = pd.DataFrame(processed_tr_pre.detach().cpu(), columns = ['OS.time', 'OS', 'age', 'stage_i', 'stage_ii', 'race_white'] + z_names)
processed_tr = processed_tr.astype(float)
processed_tr = pd.concat([patient_id_train_overall, processed_tr], axis=1)

processed_tes = pd.DataFrame(processed_tes_pre.detach().cpu(), columns = ['OS.time', 'OS', 'age', 'stage_i', 'stage_ii', 'race_white'] + z_names)
processed_tes = processed_tes.astype(float)
processed_tes = pd.concat([patient_id_test_overall, processed_tes], axis=1)

tune_id_train = get_match_id(patient_id_train_overall, patient_id_train)
tune_id_valid = get_match_id(patient_id_train_overall, patient_id_valid)
processed_tr_tune = processed_tr.iloc[tune_id_train]
processed_val_tune = processed_tr.iloc[tune_id_valid]

processed_tr.to_csv("tr_z_2omics.csv", index=False)
processed_tr_tune.to_csv("tune_tr_z_2omics.csv", index=False)
processed_val_tune.to_csv("tune_val_z_2omics.csv", index=False)
processed_tes.to_csv("tes_z_2omics.csv", index=False)

In [ ]:
print("--- %s seconds ---" % (time.time() - start_time))